In [ ]:
#For Kaggle
#date=''

#For WL data
source='WL'
date='20241118'

saveImg=False

print('done')

In [ ]:
import sys
import os
from importlib import reload
fpath = os.path.join('..//scripts')
sys.path.append(fpath)

import warnings
warnings.filterwarnings('ignore')

#loading internal scripts
import frauddetection as fd
import sourcedata as sd
import result

reload(fd)
reload(sd)
reload(result)

import pandas as pd

print('done')

# Loading raw transactions file

In [ ]:
import dataimport as di
import pandas as pd 
   
dfTrx = di.read_file('../data/raw/'+source+'export'+date+'.csv')
dfTrx.head(5)


# Pie diagram showing the imbalanced data

In [ ]:
import matplotlib.pyplot as plt

print("Pie chart fraud are very unbalanced data:")
plt.pie(dfTrx['Class'].value_counts(),autopct='%1.1f%%', labels=['Genuine','Fraud'], colors=['yellowgreen','r'])
if(saveImg):
    plt.savefig('../imgs/FD01B-piediag-transaction.png')
plt.show()


# Count plot showing the imbalanced data

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
  
# count plot on single categorical variable
sns.countplot(x ='Class', data = dfTrx)
plt.title("Fraud on credit card transactions")
plt.xlabel("0=Genuine    1=Fraudulent")
plt.ylabel("Nb transactions")
 
# Show the plot
if(saveImg):
    plt.savefig('../imgs/FD01B-count-plot-transaction.png')
plt.show()

In [ ]:
dfTrx.info()

In [ ]:
nan_count = dfTrx.isnull().sum()
print('Number of NaN values:', nan_count)

# Score Visa and Mastercard

In [ ]:
import matplotlib.pyplot as plt

dfTrx0= dfTrx[(dfTrx['card_brand']=='MCD')]
dfTrx1= dfTrx[(dfTrx['card_brand']=='MCD') &(dfTrx['Class']==1)]
dfTrx2= dfTrx[(dfTrx['card_brand']=='MCD') &(dfTrx['Class']==0)]

fig,ax = plt.subplots(2,1)

ax[0].hist(x=dfTrx2['mcd_fraud_score'], label='geniune', bins=20, color='green', alpha=0.7, rwidth=0.85)
ax[0].set_title('Score Mastercard for geniune')

ax[1].set_title('Score Mastercard for fraudulent')
ax[1].hist(x=dfTrx1['mcd_fraud_score'], label='Frandulent', bins=20, color='red', alpha=0.7, rwidth=0.85)

fig.autofmt_xdate()
plt.show()

In [ ]:
sns.boxplot(data=dfTrx,y='mcd_fraud_score', hue='Class')

In [ ]:
import seaborn as sns


sns.violinplot(x='Class', y='mcd_fraud_score', data=dfTrx0, palette=['#2f7ed8', '#d6604d'])
plt.title(f'{'mcd_fraud_score'} Distribution by Class')
plt.xlabel('Class (0 = Non-Fraud, 1 = Fraud)')
plt.ylabel('mcd_fraud_score')

plt.tight_layout()
plt.suptitle("Feature Distribution Comparison by Class", y=1.02, fontsize=16)
plt.show()

In [ ]:
import matplotlib.pyplot as plt

dfTrx0= dfTrx[(dfTrx['card_brand']=='VIS')]
dfTrx1= dfTrx[(dfTrx['card_brand']=='VIS') &(dfTrx['Class']==1)]
dfTrx2= dfTrx[(dfTrx['card_brand']=='VIS') &(dfTrx['Class']==0)]

fig,ax = plt.subplots(2,1)

ax[0].hist(x=dfTrx2['vaa_score'], label='geniune', bins=20, color='green', alpha=0.7, rwidth=0.85)
ax[0].set_title('Score Visa for geniune')

ax[1].set_title('Score Visa for fraudulent')
ax[1].hist(x=dfTrx1['vaa_score'], label='Frandulent', bins=20, color='red', alpha=0.7, rwidth=0.85)

fig.autofmt_xdate()
plt.show()

In [ ]:
sns.boxplot(data=dfTrx,y='vaa_score', hue='Class')

In [ ]:
import seaborn as sns

sns.violinplot(x='Class', y='vaa_score', data=dfTrx0, palette=['#2f7ed8', '#d6604d'])
plt.title(f'{'vaa_score'} Distribution by Class')
plt.xlabel('Class (0 = Non-Fraud, 1 = Fraud)')
plt.ylabel('vaa_score')

plt.tight_layout()
plt.suptitle("Feature Distribution Comparison by Class", y=1.02, fontsize=16)
plt.show()

In [ ]:
import numpy as np
dfTrx2 =dfTrx.copy()
dfTrx2['mcd_fraud_score'].fillna(np.mean(dfTrx2['mcd_fraud_score']), inplace=True)
dfTrx2['vaa_score'].fillna(np.mean(dfTrx2['vaa_score']), inplace=True)



dfTrx2['vaa_scoreBin']=pd.cut(dfTrx2['vaa_score'], bins=5)
iv=fd.calc_iv(dfTrx2,'vaa_scoreBin','Class',pr=0)
print(f'IV vaa_score {iv}')
result.update_features_IV_result('vaa_score',iv)

dfTrx3= dfTrx2[(dfTrx2['card_brand']=='VIS')]
dfTrx3['vaa_scoreBin']=pd.cut(dfTrx3['vaa_score'], bins=5)
iv=fd.calc_iv(dfTrx3,'vaa_scoreBin','Class',pr=0)
print(f'IV vaa_score (visa only) {iv}')
result.update_features_IV_result('vaa_score (visa only)',iv)

dfTrx2['mcd_fraud_scoreBin']=pd.cut(dfTrx2['mcd_fraud_score'], bins=5)
iv=fd.calc_iv(dfTrx2,'mcd_fraud_scoreBin','Class',pr=0)
print(f'IV mcd_fraud_score {iv}')
result.update_features_IV_result('mcd_fraud_score',iv)

dfTrx3= dfTrx2[(dfTrx2['card_brand']=='MCD')]
dfTrx3['mcd_fraud_scoreBinBin']=pd.cut(dfTrx3['mcd_fraud_score'], bins=5)
iv=fd.calc_iv(dfTrx3,'mcd_fraud_scoreBin','Class',pr=0)
print(f'IV mcd_fraud_score (mcd only) {iv}')
result.update_features_IV_result('mcd_fraud_score (mcd only)',iv)

nan_count = dfTrx.isnull().sum()
print('Number of NaN values:', nan_count)

# Issue with cluster profile

In [ ]:
# issue cluster profile
print("All transactions")
print( dfTrx.cluster_profile.value_counts())
print("-----------------------")
print("Fraudulent transactions")
print( dfTrx[dfTrx['Class']==1].cluster_profile.value_counts())
# bug almost all fraudulent transaction have cluster profile equals to UNKNOWN

# Transaction Amount

In [ ]:
# Import libraries
import seaborn as sns

# Creating plot
sns.boxplot(data=dfTrx,y='trx_amount', hue='Class')

# Show the plot
if(saveImg):
    plt.savefig('../imgs/FD01B-Amount-boxplot.png')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# An "interface" to matplotlib.axes.Axes.hist() method
plt.hist(x=dfTrx['trx_amount'], bins=75, color='#0504aa', alpha=0.7, rwidth=0.85)
plt.grid(axis='y', alpha=0.75)
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title('Amount histogram')

# Show the plot
if(saveImg):
    plt.savefig('../imgs/FD01B-Amount-histogram.png')
plt.show()


In [ ]:
import numpy as np
dfTrx['trx_amount_log10']=np.log10(1+dfTrx['trx_amount'])

# Creating plot
sns.boxplot(data=dfTrx,y='trx_amount_log10', hue='Class')

# Show the plot
if(saveImg):
    plt.savefig('../imgs/FD01B-Amount-log10-boxplot.png')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# An "interface" to matplotlib.axes.Axes.hist() method
plt.hist(x=dfTrx['trx_amount_log10'], bins=75, color='#0504aa', alpha=0.7, rwidth=0.85)
plt.grid(axis='y', alpha=0.75)
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title('Log10 Amount histogram')

# Show the plot
if(saveImg):
    plt.savefig('../imgs/FD01B-Amount-log10-histogram.png')
plt.show()


In [ ]:
import seaborn as sns

title='trx_amount'
dfTrx['temp']=dfTrx['trx_amount']

bins=5
dfTrx['amountBin']=pd.qcut(dfTrx['temp'], q=bins)
iv=fd.calc_iv(dfTrx,'amountBin','Class',pr=0)
print(f'IV qcut bins={bins} {iv}')

dfTrx['amountBin']=pd.cut(dfTrx['temp'], bins=bins)
iv=fd.calc_iv(dfTrx,'amountBin','Class',pr=0)
print(f'IV  cut bins={bins} {iv}')

sns.violinplot(x='Class', y='temp', data=dfTrx, palette=['#2f7ed8', '#d6604d'])
plt.title(f'{title} Distribution by Class')
plt.xlabel('Class (0 = Non-Fraud, 1 = Fraud)')
plt.ylabel(title)

plt.tight_layout()
plt.suptitle("Feature Distribution Comparison by Class", y=1.02, fontsize=16)
plt.show()

In [ ]:
title='trx_amount'
#dfTrx['temp']=dfTrx['trx_amount']
dfTrx['temp']=np.log10(1+dfTrx['trx_amount'])

bin_labels_5 = ['1', '2','3','4','5']
bins=5
dfTrx['amountBin']=pd.cut(dfTrx['temp'], bins=bins, labels=bin_labels_5)

#print(dfTrx['amountBin'].value_counts())

cols = ['bin', 'val']
rows_list = []
for bin in bin_labels_5:
    temp=dfTrx[(dfTrx['amountBin']==bin)]
    #print(temp['Class'].value_counts(normalize=True))
    value=0.0
    if(temp['Class'].value_counts(normalize=True)[0]!=1.0):
        value=temp['Class'].value_counts(normalize=True)[1]
    #print(value)
    dataNew = {
        'bin': bin,
        'value': value
    }
    rows_list.append(dataNew)

output = pd.DataFrame(rows_list)  

print(output.head())


plt.figure(figsize=(12, 4))
sns.barplot(data=output, x='bin', y='value', palette='viridis')
plt.title('Average Fraud Probability by bin')
plt.xlabel("bin")
plt.ylabel("Avg. Fraud Probability")
plt.xticks(range(0, 5))
plt.grid()
plt.show()

In [ ]:
import seaborn as sns

title='trx_amount_log10'
dfTrx['temp']=np.log10(1+dfTrx['trx_amount'])

bins=5
dfTrx['amountBin']=pd.qcut(dfTrx['temp'], q=bins)
iv=fd.calc_iv(dfTrx,'amountBin','Class',pr=0)
print(f'IV qcut bins={bins} {iv}')

dfTrx['amountBin']=pd.cut(dfTrx['temp'], bins=bins)
iv=fd.calc_iv(dfTrx,'amountBin','Class',pr=0)
print(f'IV  cut bins={bins} {iv}')

sns.violinplot(x='Class', y='temp', data=dfTrx, palette=['#2f7ed8', '#d6604d'])
plt.title(f'{title} Distribution by Class')
plt.xlabel('Class (0 = Non-Fraud, 1 = Fraud)')
plt.ylabel(title)

plt.tight_layout()
plt.suptitle("Feature Distribution Comparison by Class", y=1.02, fontsize=16)
plt.show()

In [ ]:
import seaborn as sns
title='trx_amount_log2'
dfTrx['temp']=np.log2(1+dfTrx['trx_amount'])

bins=5
dfTrx['amountBin']=pd.qcut(dfTrx['temp'], q=bins)
iv=fd.calc_iv(dfTrx,'amountBin','Class',pr=0)
print(f'IV qcut bins={bins} {iv}')

dfTrx['amountBin']=pd.cut(dfTrx['temp'], bins=bins)
iv=fd.calc_iv(dfTrx,'amountBin','Class',pr=0)
print(f'IV  cut bins={bins} {iv}')

sns.violinplot(x='Class', y='temp', data=dfTrx, palette=['#2f7ed8', '#d6604d'])
plt.title(f'{title} Distribution by Class')
plt.xlabel('Class (0 = Non-Fraud, 1 = Fraud)')
plt.ylabel(title)

plt.tight_layout()
plt.suptitle("Feature Distribution Comparison by Class", y=1.02, fontsize=16)
plt.show()

In [ ]:
import seaborn as sns
title='trx_amount_log1p'
dfTrx['temp']=np.log1p(dfTrx['trx_amount'])

bins=5
dfTrx['amountBin']=pd.qcut(dfTrx['temp'], q=bins)
iv=fd.calc_iv(dfTrx,'amountBin','Class',pr=0)
print(f'IV qcut bins={bins} {iv}')

dfTrx['amountBin']=pd.cut(dfTrx['temp'], bins=bins)
iv=fd.calc_iv(dfTrx,'amountBin','Class',pr=0)
print(f'IV  cut bins={bins} {iv}')

sns.violinplot(x='Class', y='temp', data=dfTrx, palette=['#2f7ed8', '#d6604d'])
plt.title(f'{title} Distribution by Class')
plt.xlabel('Class (0 = Non-Fraud, 1 = Fraud)')
plt.ylabel(title)

plt.tight_layout()
plt.suptitle("Feature Distribution Comparison by Class", y=1.02, fontsize=16)
plt.show()

In [ ]:
from scipy.stats import boxcox
import seaborn as sns

title='boxcox'
dfTrx["temp"], lam = boxcox(1+dfTrx["trx_amount"])

bins=5
dfTrx['amountBin']=pd.qcut(dfTrx['temp'], q=bins)
iv=fd.calc_iv(dfTrx,'amountBin','Class',pr=0)
print(f'IV qcut bins={bins} {iv}')

dfTrx['amountBin']=pd.cut(dfTrx['temp'], bins=bins)
iv=fd.calc_iv(dfTrx,'amountBin','Class',pr=0)
print(f'IV  cut bins={bins} {iv}')

sns.violinplot(x='Class', y='temp', data=dfTrx, palette=['#2f7ed8', '#d6604d'])
plt.title(f'{title} Distribution by Class')
plt.xlabel('Class (0 = Non-Fraud, 1 = Fraud)')
plt.ylabel(title)

plt.tight_layout()
plt.suptitle("Feature Distribution Comparison by Class", y=1.02, fontsize=16)
plt.show()

In [ ]:
from scipy.stats import yeojohnson
import seaborn as sns

title='yeojohnson'
dfTrx["temp"], lam = yeojohnson(dfTrx["trx_amount"])

bins=5
dfTrx['amountBin']=pd.qcut(dfTrx['temp'], q=bins)
iv=fd.calc_iv(dfTrx,'amountBin','Class',pr=0)
print(f'IV qcut bins={bins} {iv}')

dfTrx['amountBin']=pd.cut(dfTrx['temp'], bins=bins)
iv=fd.calc_iv(dfTrx,'amountBin','Class',pr=0)
print(f'IV  cut bins={bins} {iv}')

sns.violinplot(x='Class', y='temp', data=dfTrx, palette=['#2f7ed8', '#d6604d'])
plt.title(f'{title} Distribution by Class')
plt.xlabel('Class (0 = Non-Fraud, 1 = Fraud)')
plt.ylabel(title)

plt.tight_layout()
plt.suptitle("Feature Distribution Comparison by Class", y=1.02, fontsize=16)
plt.show()

In [ ]:
import seaborn as sns

title='square root'
dfTrx["temp"] = dfTrx["trx_amount"]**(1/2)

bins=5
dfTrx['amountBin']=pd.qcut(dfTrx['temp'], q=bins)
iv=fd.calc_iv(dfTrx,'amountBin','Class',pr=0)
print(f'IV qcut bins={bins} {iv}')

dfTrx['amountBin']=pd.cut(dfTrx['temp'], bins=bins)
iv=fd.calc_iv(dfTrx,'amountBin','Class',pr=0)
print(f'IV  cut bins={bins} {iv}')

sns.violinplot(x='Class', y='temp', data=dfTrx, palette=['#2f7ed8', '#d6604d'])
plt.title(f'{title} Distribution by Class')
plt.xlabel('Class (0 = Non-Fraud, 1 = Fraud)')
plt.ylabel(title)

plt.tight_layout()
plt.suptitle("Feature Distribution Comparison by Class", y=1.02, fontsize=16)
plt.show()

In [ ]:
import seaborn as sns

title='reverse'
dfTrx["temp"] = 1/(1+dfTrx["trx_amount"])

bins=5
dfTrx['amountBin']=pd.qcut(dfTrx['temp'], q=bins)
iv=fd.calc_iv(dfTrx,'amountBin','Class',pr=0)
print(f'IV qcut bins={bins} {iv}')

dfTrx['amountBin']=pd.cut(dfTrx['temp'], bins=bins)
iv=fd.calc_iv(dfTrx,'amountBin','Class',pr=0)
print(f'IV  cut bins={bins} {iv}')

sns.violinplot(x='Class', y='temp', data=dfTrx, palette=['#2f7ed8', '#d6604d'])
plt.title(f'{title} Distribution by Class')
plt.xlabel('Class (0 = Non-Fraud, 1 = Fraud)')
plt.ylabel(title)

plt.tight_layout()
plt.suptitle("Feature Distribution Comparison by Class", y=1.02, fontsize=16)
plt.show()

In [ ]:
import numpy as np
max = 6

print('trx_amount bin cut')
for bins in np.arange(2,max,1):
    dfTrx['amountBin']=pd.cut(dfTrx['trx_amount'], bins=bins)
    iv=fd.calc_iv(dfTrx,'amountBin','Class',pr=0)
    print(f'IV trx_local_amt_val bins={bins} {iv}')
    

print('trx_amount bin qcut')
for bins in np.arange(2,max,1):
    dfTrx['amountBin']=pd.qcut(dfTrx['trx_amount'], q=bins)
    iv=fd.calc_iv(dfTrx,'amountBin','Class',pr=0)
    print(f'IV trx_local_amt_val bins={bins} {iv}')

dfTrx['trx_amount_Log10']=np.log10(1+dfTrx['trx_amount'])
print('trx_amount log10 bin cut')
for bins in np.arange(2,max,1):
    dfTrx['amountBin']=pd.cut(dfTrx['trx_amount_Log10'], bins=bins)
    iv=fd.calc_iv(dfTrx,'amountBin','Class',pr=0)
    print(f'IV trx_local_amt_val bins={bins} {iv}')

print('trx_amount log10 bin qcut')
for bins in np.arange(2,max,1):
    dfTrx['amountBin']=pd.qcut(dfTrx['trx_amount_Log10'], q=bins)
    iv=fd.calc_iv(dfTrx,'amountBin','Class',pr=0)
    print(f'IV trx_local_amt_val bins={bins} {iv}')
result.update_features_IV_result(f'trx_local_amt_val bins={max-1}',iv)

# Reversal column

In [ ]:
dfTrx['trx_reversal']
print(dfTrx.trx_reversal.value_counts())
print('------------ fraudulent--------')
print( dfTrx[dfTrx['Class']==1].trx_reversal.value_counts())

iv =fd.calc_iv(dfTrx,'trx_reversal','Class',pr=0)
print(f'IV trx_reversal  {iv}')
result.update_features_IV_result('trx_reversal',iv)


In [ ]:
import matplotlib.pyplot as plt

print("Pie chart trx reversal")
plt.pie(dfTrx['trx_reversal'].value_counts(),autopct='%1.1f%%', labels=['NO REVERSAL','FULL REVERSAL','PARTIAL REVERSAL'], colors=['yellowgreen','r','r'])
#if(saveImg):
#    plt.savefig('../imgs/FD01B-piediag-transaction.png')
plt.show()

In [ ]:
dfTrx.trx_cnp.value_counts()

dfTrx['trx_cnp']
print(dfTrx.trx_cnp.value_counts())

print('------------ Geniune--------')
print( dfTrx[dfTrx['Class']==0].trx_cnp.value_counts())
print( dfTrx[dfTrx['Class']==0].trx_cnp.value_counts(normalize=True))
print('------------ fraudulent--------')
print( dfTrx[dfTrx['Class']==1].trx_cnp.value_counts())
print( dfTrx[dfTrx['Class']==1].trx_cnp.value_counts(normalize=True))

iv=fd.calc_iv(dfTrx,'trx_cnp','Class',pr=0)

print(f'IV trx_cnp  {iv}')
result.update_features_IV_result('trx_cnp',iv)

In [ ]:
dfTrx['ecom'] =np.where(dfTrx.card_entry_mode.isin([5,6,9]), 1,0)

iv=fd.calc_iv(dfTrx,'ecom','Class',pr=0)

print(f'IV ecom {iv}')
result.update_features_IV_result('ecom',iv)

In [ ]:
def get_seconds(date):
    return date.hour*60*24+date.minute*60+date.second

# Histogram 
# set the style 
sns.set_style('whitegrid')
# create a histogram
plt.figure(figsize=(12, 3))
dfTrx['Time']= dfTrx['trx_date_time'].apply(get_seconds)
sns.histplot( dfTrx['Time'], bins=30, kde=True, color='blue')
plt.title('Distribution of Transaction Time')
plt.xlabel("Time (in Seconds)")
plt.ylabel("Frequency")
plt.axvline(x =  dfTrx['Time'].median(), color='red', linestyle='dashed', linewidth=2, label='Median Time')
plt.axvline(x =  dfTrx['Time'].mean(), color='green', linestyle='dashed', linewidth=2, label='Mean Time')
plt.legend()
plt.show()
# LinePlot 
# create a line plot to analyze fraud over time 
plt.figure(figsize=(12, 3))
time_fruad =  dfTrx.groupby('Time')['Class'].mean().reset_index()
plt.plot(time_fruad['Time'], time_fruad['Class'], color='blue')
plt.title('Fraud Probability Over Time')
plt.xlabel("Time (in Seconds)")
plt.ylabel("Fraud Probability")
plt.grid()
plt.show()

In [ ]:
def get_hours(date):
    return date.hour


dfTrx['Hour']= dfTrx['trx_date_time'].apply(get_hours)

# caluclate the mean fraud probability by hour 
hour_fraud = dfTrx.groupby('Hour')['Class'].mean().reset_index()

# plotting the average fruad probability by hour 
plt.figure(figsize=(12, 4))
sns.barplot(data=hour_fraud, x='Hour', y='Class', palette='viridis')
plt.title('Average Fraud Probability by Hour of the Day')
plt.xlabel("Hour of the Day")
plt.ylabel("Avg. Fraud Probability")
plt.xticks(range(0, 24))
plt.grid()
plt.show()

In [ ]:
iv=fd.calc_iv(dfTrx,'Hour','Class',pr=0)
print(iv)
result.update_features_IV_result('Hour',iv)

In [ ]:
iv=fd.calc_iv(dfTrx,'ecom_indicator','Class',pr=0)
#dataIV.loc[len(dataIV)] = ["ecom_indicator", iv]
print(iv)
result.update_features_IV_result('ecom_indicator',iv)

In [ ]:
print(dfTrx.ecom_indicator.value_counts())

print(0)
print(dfTrx[dfTrx.ecom_indicator==0].Class.value_counts())
print(2)
print(dfTrx[dfTrx.ecom_indicator==2].Class.value_counts())
print(5)
print(dfTrx[dfTrx.ecom_indicator==5].Class.value_counts())
print(61)
print(dfTrx[dfTrx.ecom_indicator==61].Class.value_counts())
print(60)
print(dfTrx[dfTrx.ecom_indicator==60].Class.value_counts())
print(62)
print(dfTrx[dfTrx.ecom_indicator==62].Class.value_counts())
print(7)
print(dfTrx[dfTrx.ecom_indicator==7].Class.value_counts())
print(dfTrx[dfTrx.ecom_indicator==28].Class.value_counts())
print(dfTrx[dfTrx.ecom_indicator==4].Class.value_counts())
print(dfTrx[dfTrx.ecom_indicator==1].Class.value_counts())


In [ ]:
dfTrx['ecom_indicator2'] = np.where(dfTrx.ecom_indicator.isin([0,2,5]), dfTrx.ecom_indicator,10)


print(dfTrx.ecom_indicator2.value_counts())
iv=fd.calc_iv(dfTrx,'ecom_indicator2','Class',pr=0)
print(iv)